# 02: IBM Runtime smoke test (job mode, no hardcoded backend)

This notebook demonstrates a minimal IBM Runtime path that:
- Initializes `QiskitRuntimeService` from environment variables or saved local account.
- Discovers hardware backends with canonical filters.
- Selects a backend via `least_busy(...)` (no hardcoded `ibm_*` backend names).
- Submits a minimal `SamplerV2` job in **job mode** (not session-only).

If credentials are unavailable, the notebook prints guidance and exits cleanly to preserve a reviewer-friendly flow.

In [ ]:
from pathlib import Path
import sys

cwd = Path.cwd().resolve()
repo_root = cwd.parent if cwd.name == 'notebooks' else cwd
if str(repo_root) not in sys.path:
    sys.path.insert(0, str(repo_root))

print('Repo root on path:', repo_root)

In [ ]:
from qiskit import QuantumCircuit
from qiskit.transpiler import generate_preset_pass_manager

from src.backend_select import (
    BackendSelectionError,
    MissingCredentialsError,
    initialize_service,
    list_candidate_backends,
    select_least_busy_backend,
    summarize_backends,
)

In [ ]:
service = None

try:
    service = initialize_service()
    print('Connected to IBM Runtime service.')
except MissingCredentialsError as exc:
    print('IBM Runtime credentials are unavailable or invalid.')
    print(exc)
    print('Run local smoke notebooks: notebooks/00_local_statevector_smoke.ipynb and notebooks/01_local_aer_smoke.ipynb')
except Exception as exc:
    print('Unexpected runtime initialization error:', repr(exc))

In [ ]:
backend = None

if service is None:
    print('Skipping backend discovery because Runtime service is unavailable.')
else:
    try:
        candidates = list_candidate_backends(service, min_num_qubits=5)
        print(f'Candidate non-simulator operational backends: {len(candidates)}')
        for line in summarize_backends(candidates, limit=8):
            print(' -', line)

        backend = select_least_busy_backend(service, min_num_qubits=5)
        print('Selected least-busy backend:', backend.name)
    except BackendSelectionError as exc:
        print('Backend selection failed:', exc)

In [ ]:
if service is None or backend is None:
    print('Skipping Runtime job submission.')
else:
    try:
        from qiskit_ibm_runtime import SamplerV2

        qc = QuantumCircuit(2)
        qc.h(0)
        qc.cx(0, 1)
        qc.measure_all()

        pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
        isa_qc = pm.run(qc)

        sampler = SamplerV2(mode=backend)
        job = sampler.run([isa_qc], shots=256)

        print('Backend:', backend.name)
        print('Job ID:', job.job_id())
        print('Job status (submitted):', job.status())

        result = job.result()
        print('Job status (after result):', job.status())

        pub_result = result[0]
        measured = getattr(pub_result.data, 'meas', None)
        if measured is None:
            for attr in dir(pub_result.data):
                if attr.startswith('_'):
                    continue
                reg = getattr(pub_result.data, attr)
                if hasattr(reg, 'get_bitstrings'):
                    measured = reg
                    break

        if measured is None:
            print('Could not extract measured bitstrings from result payload.')
        else:
            bitstrings = measured.get_bitstrings()
            print('Bitstring preview:', bitstrings[:12])
            if hasattr(measured, 'get_counts'):
                print('Counts:', measured.get_counts())
    except Exception as exc:
        print('Runtime job execution failed:', repr(exc))